In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### embedding

In [14]:
word = {'hello':0 , 'world':1}

In [15]:
a = torch.tensor([word['hello']])

In [16]:
embed= nn.Embedding(2,5)

In [17]:
embed(a)

tensor([[-0.4270, -0.0896,  0.0271,  0.7321,  0.7713]],
       grad_fn=<EmbeddingBackward>)

### 压缩f 的维度从5到1

In [33]:
data = torch.rand(10 , 48  , 5) #  v ,t , f 

In [34]:
data = data.view(10, -1)    #  v , t  

In [35]:
data.shape

torch.Size([10, 240])

In [30]:
linear1 = nn.Linear(240 , 48)
output = linear1(data)
output.reshape(10,48 ,1)

### 参数标准化 uniform_

In [68]:
from torch.nn.parameter import Parameter
p1 = Parameter(torch.FloatTensor(4, 4))
p1

Parameter containing:
tensor([[0.0000e+00, 3.6893e+19, 0.0000e+00, 3.6893e+19],
        [1.2612e-44, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]], requires_grad=True)

In [69]:
p1.data.uniform_(-0.5 , 0.5)

tensor([[ 0.1941, -0.2140, -0.4319,  0.2632],
        [-0.2798,  0.2652,  0.0345, -0.4496],
        [ 0.3663, -0.2713, -0.3956, -0.0487],
        [-0.4696,  0.1995,  0.0423,  0.3965]])

In [70]:
weight = Parameter(torch.FloatTensor(4, 4))
stdv = 1. / math.sqrt(weight.size(1))
weight.data.uniform_(-stdv, stdv)

tensor([[-0.2398, -0.3741, -0.4789,  0.1677],
        [ 0.3430, -0.4839, -0.1073,  0.4375],
        [ 0.3148,  0.0528, -0.2130,  0.2135],
        [-0.3913, -0.4884, -0.3234,  0.4527]])

### 定义参数

In [74]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.param = nn.Parameter(torch.rand(3,3)) 
        # 等价与self.register_parameter('param1' ,nn.Parameter(t.randn(3, 3)))
    def forward(self,input):
        x = self.param.mm(input)
        return x 
n = Net()   

In [144]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.register_parameter('param',nn.Parameter(torch.randn(3, 3)))
    def forward(self,input):
        x = self.param.mm(input)
        return x 
n = Net()   

### cluster gcn

In [36]:
c = torch.randint(3, 5, (10,2))  # n , k
w = torch.randint(3, 5, (2,3,4)) #  k , f1, f2 

In [37]:
out = torch.mm(c,w.view(2,-1)) # n , f1*f2

In [45]:
out.view(10,3,4).shape # n , f1 , f2 

torch.Size([10, 3, 4])

In [39]:
W = nn.Parameter(torch.rand(10,2,3)) 

In [40]:
con1 = nn.Conv2d(1, 1, 16) 

In [41]:
def cw( c , w):
    """
    input:  
    output : w of clustered 
    """
    CW = []
    for i in range(c.shape[0]):
        w_i = c[i][0] * w[0] + c[i][1]*w[1] + c[i][2] *w[2] 
        CW.append(w_i)
    return CW 

In [42]:
c = torch.rand(10 ,3 )  # v , k 
w = torch.rand(3,2,2)  # v , f1, f2

### soft cluster

In [72]:
linear1 = nn.Linear(5 , 1) 

In [73]:
x = torch.randn(4,2, 3, 5)

In [74]:
linear1(x).view(4,2,3).shape

torch.Size([4, 2, 3])

In [75]:
softmax = nn.Softmax(dim=-1)

In [76]:
softmax(x).shape

torch.Size([4, 2, 3, 5])

### 矩阵间的欧式距离


In [16]:
A = torch.Tensor( [[0.1 , 0.9], [0.2,0.8]] ) 
B = torch.Tensor([[0.2 , 0.8], [0.1,0.9] ])
C = torch.Tensor([[0.1 , 0.9], [0.2,0.8] ])
C = A - B 
C

tensor([[-0.1000,  0.1000],
        [ 0.1000, -0.1000]])

In [15]:
torch.sum(C**2)

tensor(0.0400)

In [47]:
def cluster_distance(A,B):
    D = A - B
    return torch.sum( D**2) 

In [24]:
# 排列组合
import itertools
list(itertools.permutations([1,2,3,4],2))
list(itertools.combinations([1,2,3,4],2))

[(1, 2), (1, 3), (1, 4), (2, 3), (2, 4), (3, 4)]

In [54]:
A = torch.Tensor( [[0.1 , 0.9], [0.2,0.8]] ) 
B = torch.Tensor([[0.2 , 0.8], [0.1,0.9] ])
C = torch.Tensor([[0.1 , 0.9], [0.2,0.8] ])
D = torch.Tensor([[0.3 , 0.7], [0.6,0.4] ])
list(itertools.combinations([A , B , C],2))

[(tensor([[0.1000, 0.9000],
          [0.2000, 0.8000]]), tensor([[0.2000, 0.8000],
          [0.1000, 0.9000]])), (tensor([[0.1000, 0.9000],
          [0.2000, 0.8000]]), tensor([[0.1000, 0.9000],
          [0.2000, 0.8000]])), (tensor([[0.2000, 0.8000],
          [0.1000, 0.9000]]), tensor([[0.1000, 0.9000],
          [0.2000, 0.8000]]))]

In [55]:
def matrix_distance(A,B):
    D = A - B
    return torch.sum(D**2) 

In [60]:
import itertools
def cluster_distance(m_list):
    p = list(itertools.combinations(m_list,2))
    res = 0 
    for item in p:
        res_item = matrix_distance(item[0] , item[1])
        res += res_item
    return res

In [64]:
cluster_distance([A,B,C,D])

tensor(1.4000)

### dataloader 

In [86]:
import torchvision
import torchvision.transforms as transforms
train_dataset = torch.rand(50, 10 , 5)

In [87]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=10, 
                                           shuffle=True)

In [88]:
len(train_loader)

5

In [89]:
for i, data in enumerate(train_loader): 
    print(i,data.size())

0 torch.Size([10, 10, 5])
1 torch.Size([10, 10, 5])
2 torch.Size([10, 10, 5])
3 torch.Size([10, 10, 5])
4 torch.Size([10, 10, 5])


### cluster distance 

In [54]:
import itertools
def matrix_distance(A,B):
    D = A - B
    return torch.mean(D**2) 
def cluster_distance(m_list):
    p = list(itertools.combinations(m_list,2))
    res = []
    for item in p:
        res_item = matrix_distance(item[0] , item[1])
        res.append(res_item)
        res = torch.Tensor( res ) 
    return torch.mean(res)

In [55]:
A = torch.Tensor( [[0.1 , 0.9], [0.2,0.8]] ) 
B = torch.Tensor([[0.2 , 0.8], [0.1,0.9] ])

In [56]:
cluster_distance([A,B])

tensor(0.0100)

### graph to batch graph

In [64]:
graph = torch.Tensor( [ [2,3], [5,6] ])
graph.unsqueeze(0)
graph = graph.repeat(2 ,1,1) 
print(graph)

tensor([[[2., 3.],
         [5., 6.]],

        [[2., 3.],
         [5., 6.]]])


### 软分类 to 硬分类

In [5]:
cluster = torch.Tensor([[0.9,0.1],[0.2,0.8] , [0.8,0.2] , [0.7,0.3] ])
cluster

tensor([[0.9000, 0.1000],
        [0.2000, 0.8000],
        [0.8000, 0.2000],
        [0.7000, 0.3000]])

In [6]:
torch.max(cluster , 1)[0]

tensor([0.9000, 0.8000, 0.8000, 0.7000])

In [7]:
m = torch.transpose(torch.max(cluster , 1)[0].repeat(2,1),0,1)
a = torch.ones(4,2)
b = torch.zeros(4,2)
torch.where(cluster-m>=0 , a, b)

tensor([[1., 0.],
        [0., 1.],
        [1., 0.],
        [1., 0.]])

### 加batch

In [70]:
soft_cluster = torch.Tensor([[[0.9,0.1,0.1],[0.2,0.4,0.8] , [0.8,0.6,0.2] , [0.7,0.3,0.1]]])
soft_cluster.shape

torch.Size([1, 4, 3])

In [11]:
max_mat = torch.max(soft_cluster, -1)[0].unsqueeze(-1).repeat(1, 1, 3)
max_mat

tensor([[[0.9000, 0.9000, 0.9000],
         [0.8000, 0.8000, 0.8000],
         [0.8000, 0.8000, 0.8000],
         [0.7000, 0.7000, 0.7000]]])

In [12]:
out = torch.rand([1, 4 ,3])

In [13]:
torch.max(soft_cluster, -1)[0]

tensor([[0.9000, 0.8000, 0.8000, 0.7000]])

In [14]:
a = torch.ones(1,4,3)
b = torch.zeros(1,4,3)

In [15]:
hard_cluster = torch.where(soft_cluster-max_mat >=0 , a , b)
hard_cluster

tensor([[[1., 0., 0.],
         [0., 0., 1.],
         [1., 0., 0.],
         [1., 0., 0.]]])

In [16]:
hard_cluster[:, :, 0]

tensor([[1., 0., 1., 1.]])

In [17]:
out

tensor([[[0.6015, 0.1429, 0.7459],
         [0.5590, 0.0075, 0.9840],
         [0.6862, 0.3137, 0.9011],
         [0.0435, 0.7355, 0.9310]]])

In [18]:
torch.mul(out ,hard_cluster[:, :, 0].unsqueeze(-1))

tensor([[[0.6015, 0.1429, 0.7459],
         [0.0000, 0.0000, 0.0000],
         [0.6862, 0.3137, 0.9011],
         [0.0435, 0.7355, 0.9310]]])

### 构建sparse matrix

In [ ]:
import numpy as np

In [5]:
graph_2D = torch.FloatTensor((np.random.rand(4,4)>0.9).astype(np.int))
graph_2D

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

### 转置

In [4]:
x = torch.Tensor([[1,2],[3,4]])
x

tensor([[1., 2.],
        [3., 4.]])

In [5]:
torch.transpose(x, 0, 1)

tensor([[1., 3.],
        [2., 4.]])

In [9]:
x.t()

tensor([[1., 3.],
        [2., 4.]])

In [11]:
a = torch.rand(1,1,2)

In [15]:
a.squeeze(0).shape

torch.Size([1, 2])

In [16]:
a.squeeze().shape

torch.Size([2])

In [18]:
a = torch.rand(3,3, 5)
b = torch.rand(5,1)
torch.matmul(a,b).shape

torch.Size([3, 3, 1])